<!-- # Byte Pair Encoding (BPE) Tokenizer From Scratch -->
# 从头开始的字节对编码（BPE）标记器

<!-- - This is a standalone notebook implementing the popular byte pair encoding (BPE) tokenization algorithm, which is used in models like GPT-2 to GPT-4, Llama 3, etc., from scratch for educational purposes -->
- 这是一个独立的笔记本，实现了流行的字节对编码（BPE）标记化算法，该算法用于GPT-2到GPT-4， Llama 3等模型，从头开始用于教育目的
<!-- - For more details about the purpose of tokenization, please refer to [Chapter 2](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb); this code here is bonus material explaining the BPE algorithm -->
- 有关令牌化目的的更多详细信息，请参阅[第2章](../01_main-chapter-code/ch02.ipynb)；这里的代码是解释BPE算法的额外材料
<!-- - The original BPE tokenizer that OpenAI implemented for training the original GPT models can be found [here](https://github.com/openai/gpt-2/blob/master/src/encoder.py) -->
- OpenAI为训练原始GPT模型而实现的原始BPE标记器可以在[这里](https://github.com/openai/gpt-2/blob/master/src/encoder.py)找到。
<!-- - The BPE algorithm was originally described in 1994: "[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)" by Philip Gage -->
- BPE算法最初描述于1994年：Philip Gage的“[一种新的数据压缩算法](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)”
<!-- - Most projects, including Llama 3, nowadays use OpenAI's open-source [tiktoken library](https://github.com/openai/tiktoken) due to its computational performance; it allows loading pretrained GPT-2 and GPT-4 tokenizers, for example (the Llama 3 models were trained using the GPT-4 tokenizer as well) -->
- 由于其计算性能，包括Llama 3在内的大多数项目现在都使用OpenAI的开源[tiktoken库](https://github.com/openai/tiktoken)；例如，它允许加载预训练的GPT-2和GPT-4标记器（Llama 3模型也是使用GPT-4标记器进行训练的）
<!-- - The difference between the implementations above and my implementation in this notebook, besides it being is that it also includes a function for training the tokenizer (for educational purposes) -->
- 上面的实现和我在本笔记本中的实现之间的区别，除了它是，它还包括一个用于训练标记器的函数（出于学习目的）。
<!-- - There's also an implementation called [minBPE](https://github.com/karpathy/minbpe) with training support, which is maybe more performant (my implementation here is focused on educational purposes); in contrast to `minbpe` my implementation additionally allows loading the original OpenAI tokenizer vocabulary and BPE "merges" (additionally, Hugging Face tokenizers are also capable of training and loading various tokenizers; see [this GitHub discussion](https://github.com/rasbt/LLMs-from-scratch/discussions/485) by a reader who trained a BPE tokenizer on the Nepali language for more info) -->
- 还有一个名为[minBPE]（https://github.com/karpathy/minbpe）的实现，它具有培训支持，可能性能更高（我在这里的实现侧重于教育目的）；与“minbpe”相反，我的实现还允许加载原始的OpenAI标记器词汇表和BPE“合并”（此外，hug Face标记器也能够训练和加载各种标记器；参见[这个GitHub讨论](https://github.com/rasbt/LLMs-from-scratch/discussions/485)，由一位在尼泊尔语上训练BPE标记器的读者提供更多信息）

&nbsp;
<!-- # 1. The main idea behind byte pair encoding (BPE) -->
# 1. 字节对编码（BPE）背后的主要思想

<!-- - The main idea in BPE is to convert text into an integer representation (token IDs) for LLM training (see [Chapter 2](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)) -->
- BPE的主要思想是将文本转换为LLM训练的整数表示（token IDs）（参见[第2章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/bpe-overview.webp" width="800px">

&nbsp;
<!-- ## 1.1 Bits and bytes -->
## 1.1 字节对编码

<!-- - Before getting to the BPE algorithm, let's introduce the notion of bytes -->
- 在讨论BPE算法之前，让我们先介绍一下字节的概念
<!-- - Consider converting text into a byte array (BPE stands for "byte" pair encoding after all): -->
- 考虑将文本转换为字节数组（毕竟BPE代表“字节”对编码）：

In [1]:
text = "This is some text"
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'This is some text')


<!-- - When we call `list()` on a `bytearray` object, each byte is treated as an individual element, and the result is a list of integers corresponding to the byte values: -->
- 当我们在`bytearray`对象上调用`list()`时，每个字节都被视为一个单独的元素，结果是一个与字节值对应的整数列表：

In [2]:
ids = list(byte_ary)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


<!-- - This would be a valid way to convert text into a token ID representation that we need for the embedding layer of an LLM -->
- 这将是一种有效的方式，将文本转换为令牌ID表示，我们需要在LLM的嵌入层
<!-- - However, the downside of this approach is that it is creating one ID for each character (that's a lot of IDs for a short text!) -->
- 然而，这种方法的缺点是它为每个字符创建一个ID（对于一个短文本来说，这是很多ID !）
<!-- - I.e., this means for a 17-character input text, we have to use 17 token IDs as input to the LLM: -->
- 也就是说，这意味着对于17个字符的输入文本，我们必须使用17个令牌id作为LLM的输入：

In [3]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


<!-- - If you have worked with LLMs before, you may know that the BPE tokenizers have a vocabulary where we have a token ID for whole words or subwords instead of each character -->
- 如果您以前使用过llm，您可能知道BPE标记器有一个词汇表，其中我们为整个单词或子单词而不是每个字符提供令牌ID
<!-- - For example, the GPT-2 tokenizer tokenizes the same text ("This is some text") into only 4 instead of 17 tokens: `1212, 318, 617, 2420` -->
- 例如，GPT-2标记器将相同的文本（"This is some text"）标记为4个而不是17个标记： `1212, 318, 617, 2420`
<!-- - You can double-check this using the interactive [tiktoken app](https://tiktokenizer.vercel.app/?model=gpt2) or the [tiktoken library](https://github.com/openai/tiktoken): -->
- 你可以使用互动的[tiktoken应用程序](https://tiktokenizer.vercel.app/?model=gpt2)或[tiktoken库](https://github.com/openai/tiktoken):

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="800px">

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```

<!-- - Since a byte consists of 8 bits, there are 2<sup>8</sup> = 256 possible values that a single byte can represent, ranging from 0 to 255 -->
- 由于一个字节由8位组成，因此有2<sup>8</sup> = 256个可能的值，单个字节可以表示，范围从0到255
<!-- - You can confirm this by executing the code `bytearray(range(0, 257))`, which will warn you that `ValueError: byte must be in range(0, 256)`) -->
- 你可以通过执行代码`bytearray(range(0, 257))`来确认这一点，它会警告你 `ValueError: byte must be in range(0, 256)`
<!-- - A BPE tokenizer usually uses these 256 values as its first 256 single-character tokens; one could visually check this by running the following code: -->
- BPE标记器通常使用这256个值作为其前256个单字符标记；可以通过运行以下代码来直观地检查：

```python
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")
"""
prints:
0: !
1: "
2: #
...
255: �  # <---- single character tokens up to here
256:  t
257:  a
...
298: ent
299:  n
"""
```

<!-- - Above, note that entries 256 and 257 are not single-character values but double-character values (a whitespace + a letter), which is a little shortcoming of the original GPT-2 BPE Tokenizer (this has been improved in the GPT-4 tokenizer) -->
- 上面，请注意条目256和257不是单字符值，而是双字符值（一个空格+一个字母），这是原始GPT-2 BPE标记器的一个小缺点（这在GPT-4标记器中得到了改进）。

&nbsp;
<!-- ## 1.2 Building the vocabulary -->
## 1.2 建立词汇表

<!-- - The goal of the BPE tokenization algorithm is to build a vocabulary of commonly occurring subwords like `298: ent` (which can be found in *entangle, entertain, enter, entrance, entity, ...*, for example), or even complete words like -->
- BPE标记化算法的目标是建立一个常见子词的词汇表，如 `298: ent` （可以在*entangle， entertainment, enter, entrance, entity，…中找到）。*，例如)，甚至完整的单词，如

```
318: is
617: some
1212: This
2420: text
```

<!-- - The BPE algorithm was originally described in 1994: "[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)" by Philip Gage -->
- BPE算法最初描述于1994年：Philip Gage的"[一种新的数据压缩算法](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"
<!-- - Before we get to the actual code implementation, the form that is used for LLM tokenizers today can be summarized as described in the following sections. -->
- 在我们开始实际的代码实现之前，现在LLM标记器使用的形式可以总结为下面几节所描述的。

&nbsp;
<!-- ## 1.3 BPE algorithm outline -->
## 1.3 BPE算法概要

<!-- **1. Identify frequent pairs** -->
**1. 识别频繁配对**
<!-- - In each iteration, scan the text to find the most commonly occurring pair of bytes (or characters) -->
- 在每次迭代中，扫描文本以找到最常出现的字节对（或字符）

<!-- **2. Replace and record** -->
**2. 更换记录**

<!-- - Replace that pair with a new placeholder ID (one not already in use, e.g., if we start with 0...255, the first placeholder would be 256) -->
- 用一个新的占位符ID(一个尚未使用的，例如，如果我们从0开始…第一个占位符应该是256)
<!-- - Record this mapping in a lookup table -->
- 将此映射记录在查找表中
<!-- - The size of the lookup table is a hyperparameter, also called "vocabulary size" (for GPT-2, that's 50,257) -->
- 查找表的大小是一个超参数，也称为“词汇表大小”(对于GPT-2，它是 50,257)
  

<!-- **3. Repeat until no gains** -->
**3.重复，直到没有收获**

<!-- - Keep repeating steps 1 and 2, continually merging the most frequent pairs -->
- 重复步骤1和2，不断合并最频繁的配对
<!-- - Stop when no further compression is possible (e.g., no pair occurs more than once) -->
- 当无法进一步压缩时停止（例如，没有一对出现超过一次）

<!-- **Decompression (decoding)** -->
**解压 (解码)**
<!-- - To restore the original text, reverse the process by substituting each ID with its corresponding pair, using the lookup table -->
- 要恢复原始文本，可以使用查找表将每个ID替换为对应的对，从而逆转该过程



&nbsp;
<!-- ## 1.4 BPE algorithm example -->
## 1.4 BPE算法示例

<!-- ### 1.4.1 Concrete example of the encoding part (steps 1 & 2 in section 1.3) -->
### 1.4.1编码部分的具体示例（1.3节第1、2步）

<!-- - Suppose we have the text (training dataset) `the cat in the hat` from which we want to build the vocabulary for a BPE tokenizer -->
- 假设我们有一个文本（训练数据集）“帽子里的猫”，我们想要从中构建BPE标记器的词汇表

<!-- **Iteration 1** -->
**迭代1**

<!-- 1. Identify frequent pairs -->
1. 识别频繁对
  <!-- - In this text, "th" appears twice (at the beginning and before the second "e") -->
  - 在这篇文章中，“th”出现了两次（在开头和第二个“e”之前）

<!-- 2. Replace and record -->
2. 更换并记录
  <!-- - replace "th" with a new token ID that is not already in use, e.g., 256 -->
  - 将“th”替换为尚未使用的新令牌ID，例如256
  <!-- - the new text is: `<256>e cat in <256>e hat` -->
  - 新文本是： `<256>e cat in <256>e hat`
  <!-- - the new vocabulary is -->
  - 新词汇是

```
  0: ...
  ...
  256: "th"
```

<!-- **Iteration 2** -->
**迭代 2**

<!-- 1. **Identify frequent pairs** -->
1. **识别频繁对**
   <!-- - In the text `<256>e cat in <256>e hat`, the pair `<256>e` appears twice -->
   - 在文本 `<256>e cat in <256>e hat`中，对 `<256>e`出现两次

<!-- 2. **Replace and record** -->
2.  **更换并记录**
   <!-- - replace `<256>e` with a new token ID that is not already in use, for example, `257`. -->
   - 将 `<256>e` 替换为一个尚未使用的新令牌ID，例如`257`。
   <!-- - The new text is: -->
   - 新文本是：
     ```
     <257> cat in <257> hat
     ```
   <!-- - The updated vocabulary is: -->
   - 更新后的词汇是：
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     ```

<!-- **Iteration 3** -->
**迭代 3**
<!-- 1. **Identify frequent pairs**   -->
1. **识别频繁对**
   <!-- - In the text `<257> cat in <257> hat`, the pair `<257> ` appears twice (once at the beginning and once before “hat”). -->
   - 在文本 `<257> cat in <257> hat`中，对 `<257> `出现了两次（一次在开头，一次在“hat”之前）。

<!-- 2. **Replace and record**   -->
2.  **更换并记录**
   <!-- - replace `<257> ` with a new token ID that is not already in use, for example, `258`. -->
   - 将`<257> `替换为尚未使用的新令牌ID，例如`258`。
   <!-- - the new text is: -->
   - 新文本为：
     ```
     <258>cat in <258>hat
     ```
   <!-- - The updated vocabulary is: -->
   - 更新后的词汇为：
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     258: "<257> "
     ```
     
<!-- - and so forth -->
- 等等

&nbsp;
<!-- ### 1.4.2 Concrete example of the decoding part (step 3 in section 1.3) -->
### 1.4.2 解码部分的具体示例（1.3节中的步骤3）
<!-- - To restore the original text, we reverse the process by substituting each token ID with its corresponding pair in the reverse order they were introduced -->
- 为了恢复原始文本，我们通过将每个令牌 ID 替换为其对应的对（按照它们引入的相反顺序）来反转该过程
<!-- - Start with the final compressed text: `<258>cat in <258>hat` -->
- 从最终的压缩文本开始 `<258>cat in <258>hat`
<!-- -  Substitute `<258>` → `<257> `: `<257> cat in <257> hat` -->
-  替换 `<258>` → `<257> `: `<257> cat in <257> hat`
<!-- - Substitute `<257>` → `<256>e`: `<256>e cat in <256>e hat` -->
- 替换 `<257>` → `<256>e`: `<256>e cat in <256>e hat`
<!-- - Substitute `<256>` → "th": `the cat in the hat` -->
- 替换 `<256>` → "th": `the cat in the hat`

&nbsp;
<!-- ## 2. A simple BPE implementation -->
## 2. 一个简单的 BPE 实现

<!-- - Below is an implementation of this algorithm described above as a Python class that mimics the `tiktoken` Python user interface -->
- 下面是上述算法的实现，作为一个 Python 类，模仿`tiktoken` Python 用户界面
<!-- - Note that the encoding part above describes the original training step via `train()`; however, the `encode()` method works similarly (although it looks a bit more complicated because of the special token handling): -->
- 请注意，上面的编码部分通过 `train()`描述了原始训练步骤；然而，`encode()` 方法的工作原理类似（尽管由于特殊的令牌处理，它看起来有点复杂）：

<!-- 1. Split the input text into individual bytes -->
1. 将输入文本拆分为单独的字节
<!-- 2. Repeatedly find & replace (merge) adjacent tokens (pairs) when they match any pair in the learned BPE merges (from highest to lowest "rank," i.e., in the order they were learned) -->
2. 当相邻标记（对）与学习的 BPE 合并中的任何对匹配时，重复查找和替换（合并）相邻标记（对）（从最高到最低“排名”，即按照学习的顺序）
<!-- 3. Continue merging until no more merges can be applied -->
3. 继续合并，直到无法再应用合并为止
<!-- 4. The final list of token IDs is the encoded output -->
4. 最终的令牌 ID 列表是编码输出

In [3]:
from collections import Counter, deque
from functools import lru_cache
import re
import json


class BPETokenizerSimple:
    def __init__(self):
        # Maps token_id to token_str (e.g., {11246: "some"})
        self.vocab = {}
        # Maps token_str to token_id (e.g., {"some": 11246})
        self.inverse_vocab = {}
        # Dictionary of BPE merges: {(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}

        # For the official OpenAI GPT-2 merges, use a rank dict:
        #  of form {(string_A, string_B): rank}, where lower rank = higher priority
        self.bpe_ranks = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        Train the BPE tokenizer from scratch.

        Args:
            text (str): The training text.
            vocab_size (int): The desired vocabulary size.
            allowed_special (set): A set of special tokens to include.
        """

        # Preprocess: Replace spaces with "Ġ"
        # Note that Ġ is a particularity of the GPT-2 BPE implementation
        # E.g., "Hello world" might be tokenized as ["Hello", "Ġworld"]
        # (GPT-4 BPE would tokenize it as ["Hello", " world"])
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # Initialize vocab with unique characters, including "Ġ" if present
        # Start with the first 256 ASCII characters
        unique_chars = [chr(i) for i in range(256)]
        unique_chars.extend(
            char for char in sorted(set(processed_text))
            if char not in unique_chars
        )
        if "Ġ" not in unique_chars:
            unique_chars.append("Ġ")

        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # Add allowed special tokens
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # Tokenize the processed_text into token IDs
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE steps 1-3: Repeatedly find and replace frequent pairs
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # Build the vocabulary with merged tokens
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def load_vocab_and_merges_from_openai(self, vocab_path, bpe_merges_path):
        """
        Load pre-trained vocabulary and BPE merges from OpenAI's GPT-2 files.

        Args:
            vocab_path (str): Path to the vocab file (GPT-2 calls it 'encoder.json').
            bpe_merges_path (str): Path to the bpe_merges file  (GPT-2 calls it 'vocab.bpe').
        """
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # encoder.json is {token_str: id}; we want id->str and str->id
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}
    
        # Must have GPT-2's printable newline character 'Ċ' (U+010A) at id 198
        if "Ċ" not in self.inverse_vocab or self.inverse_vocab["Ċ"] != 198:
            raise KeyError("Vocabulary missing GPT-2 newline glyph 'Ċ' at id 198.")
    
        # Must have <|endoftext|> at 50256
        if "<|endoftext|>" not in self.inverse_vocab or self.inverse_vocab["<|endoftext|>"] != 50256:
            raise KeyError("Vocabulary missing <|endoftext|> at id 50256.")
    
        # Provide a convenience alias for '\n' -> 198
        # Keep printable character 'Ċ' in vocab so BPE merges keep working
        if "\n" not in self.inverse_vocab:
            self.inverse_vocab["\n"] = self.inverse_vocab["Ċ"]

        if "\r" not in self.inverse_vocab:
            if 201 in self.vocab:
                self.inverse_vocab["\r"] = 201
            else:
                raise KeyError("Vocabulary missing carriage return token at id 201.")

        # Load GPT-2 merges and store ranks
        self.bpe_ranks = {}
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            if lines and lines[0].startswith("#"):
                lines = lines[1:]
    
            rank = 0
            for line in lines:
                token1, *rest = line.strip().split()
                if len(rest) != 1:
                    continue
                token2 = rest[0]
                if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                    self.bpe_ranks[(token1, token2)] = rank
                    rank += 1
                else:
                    # Safe to skip pairs whose symbols are not in vocab
                    pass


    def encode(self, text, allowed_special=None):
        """
        Encode the input text into a list of token IDs, with tiktoken-style handling of special tokens.
    
        Args:
            text (str): The input text to encode.
            allowed_special (set or None): Special tokens to allow passthrough. If None, special handling is disabled.
    
        Returns:
            List of token IDs.
        """
    
        # ---- This section is to mimic tiktoken in terms of allowed special tokens ----
        specials_in_vocab = [
            tok for tok in self.inverse_vocab
            if tok.startswith("<|") and tok.endswith("|>")
        ]
        if allowed_special is None:
            # Nothing is allowed
            disallowed = [tok for tok in specials_in_vocab if tok in text]
            if disallowed:
                raise ValueError(f"Disallowed special tokens encountered in text: {disallowed}")
        else:
            # Some spefic tokens are allowed (e.g., we use this for <|endoftext|>)
            disallowed = [tok for tok in specials_in_vocab if tok in text and tok not in allowed_special]
            if disallowed:
                raise ValueError(f"Disallowed special tokens encountered in text: {disallowed}")
        # -----------------------------------------------------------------------------

        token_ids = []
        # If some specials are allowed, split around them and passthrough those ids
        if allowed_special is not None and len(allowed_special) > 0:
            special_pattern = "(" + "|".join(
                re.escape(tok) for tok in sorted(allowed_special, key=len, reverse=True)
            ) + ")"
    
            last_index = 0
            for match in re.finditer(special_pattern, text):
                prefix = text[last_index:match.start()]
                token_ids.extend(self.encode(prefix, allowed_special=None))  # encode prefix normally
    
                special_token = match.group(0)
                if special_token in self.inverse_vocab:
                    token_ids.append(self.inverse_vocab[special_token])
                else:
                    raise ValueError(f"Special token {special_token} not found in vocabulary.")
                last_index = match.end()
    
            text = text[last_index:]  # remainder to process normally
    
            # Extra guard for any other special literals left over
            disallowed = [
                tok for tok in self.inverse_vocab
                if tok.startswith("<|") and tok.endswith("|>") and tok in text and tok not in allowed_special
            ]
            if disallowed:
                raise ValueError(f"Disallowed special tokens encountered in text: {disallowed}")

    
        # ---- Newline and carriage return handling ----
        tokens = []
        parts = re.split(r'(\r\n|\r|\n)', text)
        for part in parts:
            if part == "":
                continue
            if part == "\r\n":
                tokens.append("\r")
                tokens.append("\n")
                continue
            if part == "\r":
                tokens.append("\r")
                continue
            if part == "\n":
                tokens.append("\n")
                continue
    
            # Normal chunk without line breaks:
            # - If spaces precede a word, prefix the first word with 'Ġ' and
            #   add standalone 'Ġ' for additional spaces
            # - If spaces trail the chunk (e.g., before a newline) add
            #   standalone 'Ġ' tokens (tiktoken produces id 220 for 'Ġ')
            pending_spaces = 0
            for m in re.finditer(r'( +)|(\S+)', part):
                if m.group(1) is not None:
                    pending_spaces += len(m.group(1))
                else:
                    word = m.group(2)
                    if pending_spaces > 0:
                        tokens.append("Ġ" + word) # one leading space
                        for _ in range(pending_spaces - 1):
                            tokens.append("Ġ")  # remaining spaces as standalone
                        pending_spaces = 0
                    else:
                        tokens.append(word)
            # Trailing spaces (no following word): add standalone 'Ġ' tokens
            for _ in range(pending_spaces):
                tokens.append("Ġ")
        # ---------------------------------------------------------------
    
        # Map tokens -> ids (BPE if needed)
        for tok in tokens:
            if tok in self.inverse_vocab:
                token_ids.append(self.inverse_vocab[tok])
            else:
                token_ids.extend(self.tokenize_with_bpe(tok))
    
        return token_ids

    def tokenize_with_bpe(self, token):
        """
        Tokenize a single token using BPE merges.

        Args:
            token (str): The token to tokenize.

        Returns:
            List[int]: The list of token IDs after applying BPE.
        """
        # Tokenize the token into individual characters (as initial token IDs)
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        # If we haven't loaded OpenAI's GPT-2 merges, use my approach
        if not self.bpe_ranks:
            can_merge = True
            while can_merge and len(token_ids) > 1:
                can_merge = False
                new_tokens = []
                i = 0
                while i < len(token_ids) - 1:
                    pair = (token_ids[i], token_ids[i + 1])
                    if pair in self.bpe_merges:
                        merged_token_id = self.bpe_merges[pair]
                        new_tokens.append(merged_token_id)
                        # Uncomment for educational purposes:
                        # print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                        i += 2  # Skip the next token as it's merged
                        can_merge = True
                    else:
                        new_tokens.append(token_ids[i])
                        i += 1
                if i < len(token_ids):
                    new_tokens.append(token_ids[i])
                token_ids = new_tokens
            return token_ids

        # Otherwise, do GPT-2-style merging with the ranks:
        # 1) Convert token_ids back to string "symbols" for each ID
        symbols = [self.vocab[id_num] for id_num in token_ids]

        # Repeatedly merge all occurrences of the lowest-rank pair
        while True:
            # Collect all adjacent pairs
            pairs = set(zip(symbols, symbols[1:]))
            if not pairs:
                break

            # Find the pair with the best (lowest) rank
            min_rank = float("inf")
            bigram = None
            for p in pairs:
                r = self.bpe_ranks.get(p, float("inf"))
                if r < min_rank:
                    min_rank = r
                    bigram = p

            # If no valid ranked pair is present, we're done
            if bigram is None or bigram not in self.bpe_ranks:
                break

            # Merge all occurrences of that pair
            first, second = bigram
            new_symbols = []
            i = 0
            while i < len(symbols):
                # If we see (first, second) at position i, merge them
                if i < len(symbols) - 1 and symbols[i] == first and symbols[i+1] == second:
                    new_symbols.append(first + second)  # merged symbol
                    i += 2
                else:
                    new_symbols.append(symbols[i])
                    i += 1
            symbols = new_symbols

            if len(symbols) == 1:
                break

        # Finally, convert merged symbols back to IDs
        merged_ids = [self.inverse_vocab[sym] for sym in symbols]
        return merged_ids

    def decode(self, token_ids):
        """
        Decode a list of token IDs back into a string.

        Args:
            token_ids (List[int]): The list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        out = []
        for tid in token_ids:
            if tid not in self.vocab:
                raise ValueError(f"Token ID {tid} not found in vocab.")
            tok = self.vocab[tid]

            # Map GPT-2 special chars back to real chars
            if tid == 198 or tok == "\n":
                out.append("\n")
            elif tid == 201 or tok == "\r":
                out.append("\r")
            elif tok.startswith("Ġ"):
                out.append(" " + tok[1:])
            else:
                out.append(tok)
        return "".join(out)

    def save_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        Save the vocabulary and BPE merges to JSON files.

        Args:
            vocab_path (str): Path to save the vocabulary.
            bpe_merges_path (str): Path to save the BPE merges.
        """
        # Save vocabulary
        with open(vocab_path, "w", encoding="utf-8") as file:
            json.dump(self.vocab, file, ensure_ascii=False, indent=2)

        # Save BPE merges as a list of dictionaries
        with open(bpe_merges_path, "w", encoding="utf-8") as file:
            merges_list = [{"pair": list(pair), "new_id": new_id}
                           for pair, new_id in self.bpe_merges.items()]
            json.dump(merges_list, file, ensure_ascii=False, indent=2)

    def load_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        Load the vocabulary and BPE merges from JSON files.

        Args:
            vocab_path (str): Path to the vocabulary file.
            bpe_merges_path (str): Path to the BPE merges file.
        """
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            self.vocab = {int(k): v for k, v in loaded_vocab.items()}
            self.inverse_vocab = {v: int(k) for k, v in loaded_vocab.items()}

        # Load BPE merges
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            merges_list = json.load(file)
            for merge in merges_list:
                pair = tuple(merge["pair"])
                new_id = merge["new_id"]
                self.bpe_merges[pair] = new_id

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if not pairs:
            return None

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced

<!-- - There is a lot of code in the `BPETokenizerSimple` class above, and discussing it in detail is out of scope for this notebook, but the next section offers a short overview of the usage to understand the class methods a bit better -->
- 上面的“BPETokenizerSimple”类中有很多代码，详细讨论它超出了本笔记本的范围，但下一节提供了用法的简短概述，以便更好地理解类方法

<!-- ## 3. BPE implementation walkthrough -->
## 3. BPE 实施演练

<!-- - In practice, I highly recommend using [tiktoken](https://github.com/openai/tiktoken) as my implementation above focuses on readability and educational purposes, not on performance -->
- 在实践中，我强烈建议使用 [tiktoken](https://github.com/openai/tiktoken)，因为我上面的实现侧重于可读性和学习目的，而不是性能
<!-- - However, the usage is more or less similar to tiktoken, except that tiktoken does not have a training method -->
- 不过用法或多或少和tiktoken类似，只不过tiktoken没有训练方法
<!-- - Let's see how my `BPETokenizerSimple` Python code above works by looking at some examples below (a detailed code discussion is out of scope for this notebook) -->
- 让我们通过查看下面的一些示例来了解上面的`BPETokenizerSimple`Python 代码是如何工作的（详细的代码讨论超出了本笔记本的范围）

<!-- ### 3.1 Training, encoding, and decoding -->
### 3.1 训练、编码和解码

<!-- - First, let's consider some sample text as our training dataset: -->
- 首先，让我们考虑一些示例文本作为我们的训练数据集：

In [4]:
import os
import requests

def download_file_if_absent(url, filename, search_dirs):
    for directory in search_dirs:
        file_path = os.path.join(directory, filename)
        if os.path.exists(file_path):
            print(f"{filename} already exists in {file_path}")
            return file_path

    target_path = os.path.join(search_dirs[0], filename)
    try:
        response = requests.get(url, stream=True, timeout=60)
        response.raise_for_status()
        with open(target_path, "wb") as out_file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    out_file.write(chunk)
        print(f"Downloaded {filename} to {target_path}")
    except Exception as e:
        print(f"Failed to download {filename}. Error: {e}")

    return target_path


verdict_path = download_file_if_absent(
    url=(
         "https://raw.githubusercontent.com/rasbt/"
         "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
         "the-verdict.txt"
    ),
    filename="the-verdict.txt",
    search_dirs=["ch02/01_main-chapter-code/", "../01_main-chapter-code/", "."]
)

with open(verdict_path, "r", encoding="utf-8") as f: # added ../01_main-chapter-code/
    text = f.read()

the-verdict.txt already exists in ../01_main-chapter-code/the-verdict.txt


<!-- - Next, let's initialize and train the BPE tokenizer with a vocabulary size of 1,000 -->
- 接下来，我们初始化并训练词汇量为 1,000 的 BPE 分词器
<!-- - Note that the vocabulary size is already 256 by default due to the byte values discussed earlier, so we are only "learning" 744 vocabulary entries (if we consider the `<|endoftext|>` special token and the `Ġ` whitespace token; so, that's 742 to be precise) -->
- 请注意，由于前面讨论的字节值，默认情况下词汇表大小已经为 256，因此我们仅“学习”744 个词汇表条目（如果我们考虑 `<|endoftext|>` 特殊标记和 `Ġ` 空白标记；因此，准确地说是 742 个）
<!-- - For comparison, the GPT-2 vocabulary is 50,257 tokens, the GPT-4 vocabulary is 100,256 tokens (`cl100k_base` in tiktoken), and GPT-4o uses 199,997 tokens (`o200k_base` in tiktoken); they have all much bigger training sets compared to our simple example text above -->
- 作为比较，GPT-2 词汇表为 50,257 个标记，GPT-4 词汇表为 100,256 个标记（tiktoken 中的`cl100k_base` ），GPT-4o 使用 199,997 个标记（tiktoken 中的`o200k_base`）；与我们上面的简单示例文本相比，它们都有更大的训练集

In [5]:
tokenizer = BPETokenizerSimple()
tokenizer.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})

<!-- - You may want to inspect the vocabulary contents (but note it will create a long list) -->
- 您可能想检查词汇内容（但请注意，它将创建一个很长的列表）

In [8]:
# print(tokenizer.vocab)
print(len(tokenizer.vocab))

1000


<!-- - This vocabulary is created by merging 742 times (`= 1000 - len(range(0, 256)) - len(special_tokens) - "Ġ" = 1000 - 256 - 1 - 1 = 742`) -->
- 该词汇表是通过合并 742 次创建的 (`= 1000 - len(range(0, 256)) - len(special_tokens) - "Ġ" = 1000 - 256 - 1 - 1 = 742`)

In [9]:
print(len(tokenizer.bpe_merges))

742


<!-- - This means that the first 256 entries are single-character tokens -->
- 这意味着前 256 个条目是单字符标记

<!-- - Next, let's use the created merges via the `encode` method to encode some text: -->
- 接下来，让我们通过`encode` 方法使用创建的合并来编码一些文本：

In [10]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [11]:
input_text = "Jack embraced beauty through art and life.<|endoftext|> "
token_ids = tokenizer.encode(input_text, allowed_special={"<|endoftext|>"})
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 257, 256]


In [12]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 56
Number of token IDs: 22


<!-- - From the lengths above, we can see that a 42-character sentence was encoded into 20 token IDs, effectively cutting the input length roughly in half compared to a character-byte-based encoding -->
- 从上面的长度中，我们可以看到一个 42 个字符的句子被编码为 20 个 token ID，与基于字符字节的编码相比，有效地将输入长度大约减少了一半

<!-- - Note that the vocabulary itself is used in the `decode()` method, which allows us to map the token IDs back into text: -->
- 请注意，词汇表本身在`decode()`方法中使用，它允许我们将标记 ID 映射回文本：

In [13]:
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46, 257, 256]


In [14]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life.<|endoftext|> 


<!-- - Iterating over each token ID can give us a better understanding of how the token IDs are decoded via the vocabulary: -->
- 迭代每个令牌 ID 可以让我们更好地理解令牌 ID 如何通过词汇表进行解码：

In [15]:
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

424 -> Jack
256 ->  
654 -> em
531 -> br
302 -> ac
311 -> ed
256 ->  
296 -> be
97 -> a
465 -> ut
121 -> y
595 ->  through
841 ->  ar
116 -> t
287 ->  a
466 -> nd
256 ->  
326 -> li
972 -> fe
46 -> .
257 -> <|endoftext|>
256 ->  


<!-- - As we can see, most token IDs represent 2-character subwords; that's because the training data text is very short with not that many repetitive words, and because we used a relatively small vocabulary size -->
- 正如我们所看到的，大多数令牌 ID 代表 2 个字符的子词；这是因为训练数据文本非常短，没有那么多重复单词，而且我们使用的词汇量相对较小

<!-- - As a summary, calling `decode(encode())` should be able to reproduce arbitrary input texts: -->
- 总而言之，调用`decode(encode())` 应该能够重现任意输入文本：

In [16]:
tokenizer.decode(
    tokenizer.encode("This is some text.")
)

'This is some text.'

In [17]:
tokenizer.decode(
    tokenizer.encode("This is some text with \n newline characters.")
)

'This is some text with \n newline characters.'

<!-- ### 3.2 Saving and loading the tokenizer -->
### 3.2 保存和加载分词器

<!-- - Next, let's look at how we can save the trained tokenizer for reuse later: -->
- 接下来，让我们看看如何保存经过训练的分词器以供以后重用：

In [18]:
# Save trained tokenizer
tokenizer.save_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

In [19]:
# Load tokenizer
tokenizer2 = BPETokenizerSimple()
tokenizer2.load_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

<!-- - The loaded tokenizer should be able to produce the same results as before: -->
- 加载的分词器应该能够产生与以前相同的结果：

In [20]:
print(tokenizer2.decode(token_ids))

Jack embraced beauty through art and life.<|endoftext|> 


In [21]:
tokenizer2.decode(
    tokenizer2.encode("This is some text with \n newline characters.")
)

'This is some text with \n newline characters.'

&nbsp;
<!-- ### 3.3 Loading the original GPT-2 BPE tokenizer from OpenAI -->
### 3.3 从 OpenAI 加载原始 GPT-2 BPE 标记器

<!-- - Finally, let's load OpenAI's GPT-2 tokenizer files -->
- 最后，让我们加载 OpenAI 的 GPT-2 分词器文件

In [23]:
# Download files if not already present in this directory

# Define the directories to search and the files to download
search_directories = ["ch02/02_bonus_bytepair-encoder/gpt2_model/", "../02_bonus_bytepair-encoder/gpt2_model/", "."]

files_to_download = {
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe": "vocab.bpe",
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json": "encoder.json"
}

# Ensure directories exist and download files if needed
paths = {}
for url, filename in files_to_download.items():
    paths[filename] = download_file_if_absent(url, filename, search_directories)

vocab.bpe already exists in ../02_bonus_bytepair-encoder/gpt2_model/vocab.bpe
encoder.json already exists in ../02_bonus_bytepair-encoder/gpt2_model/encoder.json


<!-- - Next, we load the files via the `load_vocab_and_merges_from_openai` method: -->
- 接下来，我们通过`load_vocab_and_merges_from_openai`方法加载文件：

In [24]:
tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path=paths["encoder.json"], bpe_merges_path=paths["vocab.bpe"]
)

<!-- - The vocabulary size should be `50257` as we can confirm via the code below: -->
- 词汇量大小应为“50257”，我们可以通过以下代码确认：

In [25]:
len(tokenizer_gpt2.vocab)

50257

<!-- - We can now use the GPT-2 tokenizer via our `BPETokenizerSimple` object: -->
- 现在，我们可以通过 `BPETokenizerSimple`对象使用 GPT-2 分词器：

In [26]:
input_text = "This is some text"
token_ids = tokenizer_gpt2.encode(input_text)
print(token_ids)

[1212, 318, 617, 2420]


In [27]:
print(tokenizer_gpt2.decode(token_ids))

This is some text


<!-- - You can double-check that this produces the correct tokens using the interactive [tiktoken app](https://tiktokenizer.vercel.app/?model=gpt2) or the [tiktoken library](https://github.com/openai/tiktoken): -->
- 您可以使用交互式 [tiktoken 应用程序](https://tiktokenizer.vercel.app/?model=gpt2) 或 [tiktoken 库](https://github.com/openai/tiktoken),仔细检查这会产生正确的令牌：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="800px">

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# prints [1212, 318, 617, 2420]
```


&nbsp;
<!-- # 4. Conclusion -->
# 4. 结论

<!-- - That's it! That's how BPE works in a nutshell, complete with a training method for creating new tokenizers or loading the GPT-2 tokenizer vocabular and merges from the original OpenAI GPT-2 model -->
- 就是这样！简而言之，这就是 BPE 的工作原理，包括用于创建新分词器或加载 GPT-2 分词器词汇并与原始 OpenAI GPT-2 模型合并的训练方法
<!-- - I hope you found this brief tutorial useful for educational purposes; if you have any questions, please feel free to open a new Discussion [here](https://github.com/rasbt/LLMs-from-scratch/discussions/categories/q-a) -->
- 我希望您发现这个简短的教程对教育目的有用；如果您有任何疑问，请随时在[此处](https://github.com/rasbt/LLMs-from-scratch/discussions/categories/q-a)打开新的讨论
<!-- - For a performance comparison with other tokenizer implementations, please see [this notebook](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) -->
- 有关与其他分词器实现的性能比较，请参阅[本笔记本](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb)